# Simulations-App
---
Das Modul "Simulations-App" ist das Kernstück der App, das durch die Kombination von zwei Hauptkomponenten, dem Simulator und dem Machine Learning-Teil, eine detaillierte Analyse und Vorhersage verschiedener Aspekte des Stromverbrauchs, der Stromerzeugung und der Strompreise ermöglicht. Der Simulator erlaubt es den Nutzern, den Stromverbrauch, die Stromerzeugung und die Strompreise für verschiedene Nutzertypen zu simulieren. Der Teil des maschinellen Lernens nutzt historische Daten, um akkurate Vorhersagen zu treffen.

### Simulator
---
Diese Komponente ist dafür zuständig, verschiedene Aspekte des Stromverbrauchs, der Stromerzeugung und der Stromkosten basierend auf unterschiedlichen Benutzereingaben zu simulieren. Sie hilft bei der Analyse des Stromverbrauchs, der Erzeugung und der Kosten für verschiedene Benutzertypen wie Verbraucher, Prosumer und Flexumer.

#### Funktionsweise

In diesem Modul werden die folgenden Funktionen verwendet:

1. `load_data()`: Diese Funktion lädt Daten aus einer Excel-Datei und überprüft, ob die Datei existiert. Falls ja, werden relevante Spalten eingelesen und zurückgegeben.


2. `simulate_power_consumption()`: Simuliert den Stromverbrauch basierend auf dem jährlichen Verbrauch und dem SLP-Basisprozentwert. Eine zufällige Variation wird hinzugefügt, um die Verbrauchswerte realitätsnäher zu gestalten.


3. `simulate_power_production()`: Simuliert die Stromerzeugung basierend auf der PV-Kapazität (Photovoltaik). Je nach Kapazität wird die Erzeugung angepasst und eine zufällige Variation hinzugefügt.


4. `simulate_power_price()`: Simuliert die Strompreise basierend auf dem jährlichen Verbrauch, dem aktuellen Tarif und dem Grundpreis. Es berechnet feste und variable Stromtarife sowie die Marktstrompreise.


5. `simulate_grid_consumption()`: Simuliert den Netzverbrauch basierend auf dem Benutzertyp (Consumer, Prosumer, Flexumer). Je nach Typ wird der Netzverbrauch entsprechend angepasst.


6. `simulate_power_production_usage()`: Simuliert die Nutzung der Stromerzeugung basierend auf dem Verbrauch. Diese Funktion stellt sicher, dass die Erzeugung so weit wie möglich genutzt wird, bevor Netzstrom verwendet wird.


7. `simulate_battery_usage()`: Simuliert die Nutzung eines Stromspeichers (Batterie) basierend auf der Batteriekapazität. Es wird berücksichtigt, ob überschüssige Produktion gespeichert oder Verbrauch aus der Batterie gedeckt werden kann.


8. `run_simulator()`: Dies ist die Hauptfunktion, die die Simulation basierend auf den Benutzereingaben ausführt. Sie kombiniert alle oben genannten Funktionen, um den gesamten Prozess zu simulieren, einschließlich Verbrauch, Erzeugung, Batterieeinsatz und Strompreise.

#### Quellcode

In [2]:
import os
import pandas as pd
import random
import numpy as np


df = None
data = None

def load_data():
    global df
    excel_file_path = 'SGS/Output-Data/ETL-Prozess_Ergebnisse.xlsx'
    
    # Überprüfen, ob die Excel-Datei existiert
    if os.path.exists(excel_file_path):
        # Lesen der relevanten Spalten aus der Excel-Datei
        df = pd.read_excel(excel_file_path, usecols=['Zeitstempel', 'Jahr', 'Monat', 'Tag', 'Stunde', 'Quartal', 'Kalenderwoche', 'Tag des Jahres', 'Wochentag','SLP-Basisprozentwert', 'PV-1kWp', 'Strommarktpreis [€/kWh]'], nrows=35040)
        data = df[['Zeitstempel','Jahr', 'Monat', 'Tag', 'Stunde', 'Quartal', 'Kalenderwoche', 'Tag des Jahres', 'Wochentag', 'SLP-Basisprozentwert']].copy()
        return data
    else:
        print(f"Error: Datei konnte nicht gefunden werden {excel_file_path}")
        return None

# Simuliert den Stromverbrauch basierend auf dem jährlichen Verbrauch und dem SLP-Basisprozentwert
def simulate_power_consumption(data, annual_consumption, tariff_input):
    try:
        annual_consumption = float(annual_consumption)
        # Berechnen des Stromverbrauchs basierend auf dem jährlichen Verbrauch und dem SLP-Basisprozentwert
        power_consumption = data['SLP-Basisprozentwert'] * annual_consumption / 100
        variation = random.uniform(0.001, 0.009)
        power_consumption *= (1 + variation)
        data['Stromverbrauch [kWh]'] = power_consumption
        return True
    except Exception as e:
        print(f"Fehler bei der Simulation des Stromverbrauchs: {e}")
        return False

# Simuliert die Stromerzeugung basierend auf der PV-Kapazität
def simulate_power_production(data, pv_capacity):
    try:  
        if pv_capacity == 'PV-1kWp':
            # Anpassen der PV-Erzeugungswerte um eine zufällige Variation
            adjusted_pv_1kw = df['PV-1kWp'] * np.random.uniform(0.999, 1.001)
            power_production = adjusted_pv_1kw
            data['Stromerzeugung [kWh]'] = power_production
        elif pv_capacity in ['PV-5kWp', 'PV-10kWp', 'PV-15kWp', 'PV-20kWp', 'PV-25kWp', 'PV-30kWp']:
            # Berechnen der PV-Erzeugung für verschiedene Kapazitäten
            adjusted_pv_1kw = df['PV-1kWp'] * np.random.uniform(0.999, 1.001)
            scaling_factor = int(pv_capacity.split('-')[1].replace('kWp', ''))
            power_production = adjusted_pv_1kw * scaling_factor
            data['Stromerzeugung [kWh]'] = power_production
        
        return simulate_power_production_usage(data)
    except Exception as e:
        print(f"Fehler bei der Simulation der Stromerzeugung: {e}")
        return False

# Simuliert die Strompreise basierend auf dem jährlichen Verbrauch, dem aktuellen Tarif und dem Grundpreis
def simulate_power_price(data, annual_consumption, current_tariff, current_base_price):
    global df 
    annual_consumption = float(annual_consumption)
    current_tariff = float(current_tariff)
    current_base_price = float(current_base_price)
        
    # Berechnen der festen und variablen Stromtarife
    fixed_tariff_values = (data['Lasten Netz [kWh]'] * (current_tariff/100)) + ((current_base_price * 12) / 35040)
    variable_tariff_values = (data['Lasten Netz [kWh]'] * df['Strommarktpreis [€/kWh]']) + ((current_base_price * 12) / 35040)
        
    data['Fixer Stromtarif [€/kWh]'] = fixed_tariff_values
    data['Dynamischer Stromtarif [€/kWh]'] = variable_tariff_values
    data['Strommarktpreis [€/kWh]'] = df['Strommarktpreis [€/kWh]']
    return True 

# Simuliert den Netzverbrauch basierend auf dem Benutzertyp (Verbraucher, Prosumer, Flexumer)
def simulate_grid_consumption(data, user_type):
    # Initialisieren der Netzlast mit 0
    data['Lasten Netz [kWh]'] = 0.0
    
    if user_type == 'consumer':
        # Verbraucher verbrauchen direkt aus dem Netz
        data['Lasten Netz [kWh]'] = data['Stromverbrauch [kWh]']
    else:
        for index, row in data.iterrows():
            consumption = row['Stromverbrauch [kWh]']
            
            if user_type == 'prosumer':
                # Prosumer verbrauchen zuerst Eigenproduktion, dann Netzstrom
                lasten = consumption - data.at[index, 'Lasten Stromerzeugung [kWh]']
                data.at[index,'Lasten Netz [kWh]'] =  lasten if lasten > 0 else 0.0
            if user_type == 'flexumer':
                # Flexumer nutzen zuerst Eigenproduktion und Batterie, dann Netzstrom
                lasten = consumption - data.at[index, 'Lasten Stromerzeugung [kWh]'] - data.at[index, 'Lasten Stromspeicher [kWh]']
                data.at[index,'Lasten Netz [kWh]'] = lasten if lasten > 0 else 0.0
    return True

# Simuliert die Nutzung der Stromerzeugung basierend auf dem Verbrauch
def simulate_power_production_usage(data):
    try:
        # Initialisieren der Lasten Stromerzeugung mit 0
        data['Lasten Stromerzeugung [kWh]'] = 0.0
        for index, row in data.iterrows():
            consumption = row['Stromverbrauch [kWh]']
            production = row['Stromerzeugung [kWh]']
            if production == 0:
                data.at[index, 'Lasten Stromerzeugung [kWh]'] = 0.0
            else:
                # Nutzung der Produktion basierend auf dem Verbrauch
                lasten = consumption if (production-consumption) >= 0 else production
                data.at[index, 'Lasten Stromerzeugung [kWh]'] = lasten
        return True
    except Exception as e:
        print(f"Fehler bei der Simulation Lasten Stromerzeugung: {e}")
        return False

# Simuliert die Nutzung eines Stromspeichers (Batterie) basierend auf der Batteriekapazität
def simulate_battery_usage(data, battery_capacity):
    try:
        capacity_mapping = {
            '2,5 kWh': 2.5,
            '5,1 kWh': 5.1,
            '7,7 kWh': 7.7,
            '10,2 kWh': 10.2,
            '12,8 kWh': 12.8
        }

        batt_capacity = capacity_mapping.get(battery_capacity, 0.0)

        battery_capacity = float(batt_capacity)

        data['Stromspeicher [kWh]'] = 0.0
        data['Lasten Stromspeicher [kWh]'] = 0.0
        
        batt_15mins_peak = batt_capacity / 4
     
        for index, row in data.iterrows():
            consumption = row['Stromverbrauch [kWh]']
            production = row['Stromerzeugung [kWh]']
            if production == 0:
                # Verbrauch aus der Batterie wenn keine Produktion stattfindet
                batt_consumption = consumption if consumption <= batt_15mins_peak else batt_15mins_peak
                if batt_consumption <= battery_capacity:
                    data.at[index, 'Stromspeicher [kWh]'] = battery_capacity - batt_consumption 
                    data.at[index, 'Lasten Stromspeicher [kWh]'] = consumption
                    battery_capacity -= batt_consumption
                else:
                    data.at[index, 'Stromspeicher [kWh]'] = 0.0
                    data.at[index, 'Lasten Stromspeicher [kWh]'] = battery_capacity
                    battery_capacity = 0.0
            else:
                if (production - consumption) > 0:
                    # Speichern überschüssiger Produktion in der Batterie
                    production_rest = batt_15mins_peak if (production - consumption) >= batt_15mins_peak else (production - consumption)
                    if (data.at[index - 1, 'Stromspeicher [kWh]'] + production_rest) <= batt_capacity:
                        data.at[index, 'Stromspeicher [kWh]'] = battery_capacity + production_rest
                        battery_capacity += production_rest
                    else:
                        data.at[index, 'Stromspeicher [kWh]'] = batt_capacity 
                        battery_capacity = batt_capacity
                    data.at[index, 'Lasten Stromspeicher [kWh]'] = 0.0
                else:
                    # Verbrauch aus der Batterie bei geringer Produktion
                    batt_consumption = (consumption - production) if (consumption - production) <= batt_15mins_peak else batt_15mins_peak
                    if batt_consumption <= battery_capacity:
                        data.at[index, 'Stromspeicher [kWh]'] = battery_capacity - batt_consumption
                        data.at[index, 'Lasten Stromspeicher [kWh]'] = batt_consumption
                        battery_capacity -= batt_consumption
                    else:
                        data.at[index, 'Stromspeicher [kWh]'] = 0.0
                        data.at[index, 'Lasten Stromspeicher [kWh]'] = battery_capacity
                        battery_capacity = 0.0
                        
        return True
    except Exception as e:
        print(f"Fehler bei der Simulation der Batterienutzung: {e}")
        return False

# Hauptfunktion zum Ausführen der Simulation basierend auf den Benutzereingaben
def run_simulator(annual_input, tariff_input, base_input, pv_capacity, user_type, battery_capacity):
    # Simulation des Stromverbrauchs basierend auf dem jährlichen Verbrauch und dem Tarif
    simulate_power_consumption(data, annual_input, tariff_input)
    if user_type != 'consumer':
        # Simulation der Stromerzeugung bei Prosumer und Flexumer
        simulate_power_production(data, pv_capacity)
    if user_type == 'flexumer':
        # Simulation der Batterienutzung bei Flexumer
        simulate_battery_usage(data, battery_capacity)
    # Simulation des Netzverbrauchs
    simulate_grid_consumption(data, user_type)
    # Simulation der Strompreise
    simulate_power_price(data, annual_input, tariff_input, base_input)

### Machine Learning
---
Diese Komponente zielt darauf ab, maschinelle Lernmodelle zu trainieren und zu bewerten, um den Stromverbrauch und die Strompreise vorherzusagen. Sie verwendet historische Daten, um Regressionsmodelle mithilfe des XGBoost-Algorithmus zu trainieren. Darüber hinaus bietet sie Funktionen zum Aufteilen der Daten in Trainings- und Testsets sowie zum Generieren zukünftiger Vorhersagen.

#### Funktionsweise

In diesem Modul werden die folgenden Funktionen verwendet:

1. `load_historical_data()`: Diese Funktion lädt historische Daten.


2. `select_features_and_target_variables()`: Diese Funktion wählt die Merkmale und Zielvariablen aus dem Datensatz aus. Die Merkmale umfassen die Tageszeit, den Tag des Monats, den Wochentag usw., während die Zielvariablen den Stromverbrauch und den Strompreis umfassen.


3. `split_training_data_test_by_date()`: Diese Funktion teilt die Daten basierend auf einem bestimmten Datum in Trainings- und Testsets auf. Daten vor dem Trenndatum werden zum Trainieren des Modells verwendet, während Daten danach zur Bewertung der Modellleistung dienen.


4. `split_training_test_data()`: Diese Funktion führt eine zufällige Aufteilung der Daten in Trainings- und Testsets durch. Sie ermöglicht die Festlegung der Größe des Testsets und des zufälligen Zustands, um die Reproduzierbarkeit der Ergebnisse sicherzustellen.


5. `split_and_randomize_data()`: Diese Funktion kombiniert die Aufteilung nach Datum und die zufällige Aufteilung der Daten. Sie enthält auch eine zusätzliche Aufteilung, um ein Validierungsset zu erstellen. Dies bietet eine robustere Möglichkeit, die Leistung des Modells zu bewerten.


6. `train_model_XGBoost()`: Diese Funktion trainiert ein Regressionsmodell mithilfe des XGBoost-Algorithmus. Sie passt das Modell an die Trainingsdaten an und bewertet seine Leistung auf dem Testset.


7. `evaluate_models()`: Diese Funktion bewertet die Leistung der Modelle, indem sie den mittleren quadratischen Fehler (MSE) zwischen den Vorhersagen und den tatsächlichen Werten für den Stromverbrauch und die Strompreise berechnet.


8. `create_and_train_model()`: Diese Funktion koordiniert den Prozess des Modelltrainings. Sie teilt die Daten in Trainings- und Testsets unter Verwendung verschiedener Strategien auf, trainiert Modelle für jede Strategie und wählt das Modell mit dem geringsten durchschnittlichen MSE für zukünftige Vorhersagen aus.


9. `make_predictions_2024()`: Diese Funktion generiert Vorhersagen für das Jahr 2024 mithilfe der trainierten Modelle. Sie erstellt einen Zeitraum für das gesamte Jahr 2024 und verwendet die entsprechenden Merkmale, um den Stromverbrauch und die Strompreise vorherzusagen.


10. `run_machine_learning()`: Diese ist die Hauptfunktion, die den gesamten Prozess des maschinellen Lernens durchführt. Sie lädt die historischen Daten, trainiert die Modelle und generiert Vorhersagen für das Jahr 2024.

#### Quellcode

In [3]:
import os
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Funktion zum Laden der historischen Daten
def load_historical_data(data):
    if data is not None:
        df_historical = data[['Zeitstempel', 'Stunde', 'Tag', 'Wochentag', 'Tag des Jahres', 'Kalenderwoche', 'Monat', 'Quartal', 'Jahr', 'Lasten Netz [kWh]', 'Strommarktpreis [€/kWh]']].copy()
        df_historical['Zeitstempel'] = pd.to_datetime(df_historical['Zeitstempel'])
        df_historical.set_index('Zeitstempel', inplace=True)
        return df_historical
    else:
        print("Error: Der eingegebene DataFrame ist None")
        return None

# Funktion zur Auswahl der Merkmale und Zielvariablen
def select_features_and_target_variables(data):
    X_features = data[['Stunde', 'Tag', 'Wochentag', 'Tag des Jahres', 'Kalenderwoche', 'Monat', 'Quartal', 'Jahr']]
    y_consumption = data['Lasten Netz [kWh]']
    y_price = data['Strommarktpreis [€/kWh]']
    
    return X_features, y_consumption, y_price

# Funktion zur Aufteilung der Trainingsdaten und Testdaten nach Datum
def split_training_data_test_by_date(X, y, split_date='2023-09-30'):
    # Umwandlung des Trennungsdatums in das Datetime-Format
    split_date = pd.to_datetime(split_date)
     
    # Filtern der Daten für das Jahr 2023
    X_2023 = X[X.index.year == 2023]
    y_2023 = y[X.index.year == 2023]
    
    # Aufteilen der Daten in Trainings- und Testmengen
    X_train = X_2023.loc[X_2023.index < split_date]
    X_test = X_2023.loc[X_2023.index >= split_date]
    y_train = y_2023.loc[y_2023.index < split_date]
    y_test = y_2023.loc[y_2023.index >= split_date]
    
    return X_train, X_test, y_train, y_test

# Funktion zur zufälligen Aufteilung der Daten in Trainings- und Testmengen
def split_training_test_data(X, y, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

# Kombination der beiden Methoden: erst zeitliche, dann zufällige Aufteilung
def split_and_randomize_data(X, y, split_date='2023-09-30', test_size=0.2, validation_size=0.2, random_state=42):
    # Umwandlung des Trennungsdatums in das Datetime-Format
    split_date = pd.to_datetime(split_date)
    
    # Filtern der Daten für das Jahr 2023
    X_2023 = X[X.index.year == 2023]
    y_2023 = y[X.index.year == 2023]
    
    # Aufteilen der Daten in Trainings- und Testmengen basierend auf dem Datum
    X_train_temp = X_2023.loc[X_2023.index < split_date]
    X_test = X_2023.loc[X_2023.index >= split_date]
    y_train_temp = y_2023.loc[y_2023.index < split_date]
    y_test = y_2023.loc[y_2023.index >= split_date]
    
    # Zufällige Aufteilung innerhalb der temporären Untergruppen
    X_train, X_val, y_train, y_val = train_test_split(X_train_temp, y_train_temp, test_size=validation_size, random_state=random_state)
    
    return X_train, X_val, X_test, y_train, y_val, y_test

# Funktion zur Modellierung mit XGBoost
def train_model_XGBoost(X_train, X_test, y_train, y_test):
    model = xgb.XGBRegressor(base_score=0.5, booster='gbtree',    
                             n_estimators=1000,
                             early_stopping_rounds=50,
                             objective='reg:squarederror',
                             max_depth=3,
                             learning_rate=0.01)
    model.fit(X_train, y_train,
              eval_set=[(X_train, y_train), (X_test, y_test)],
              verbose=100)
    
    return model

# Funktion zur Bewertung der Modelle
def evaluate_models(y_test_con, y_pred_con, y_test_price, y_pred_price):
   # Bewertung der Modelle
    mse_con = mean_squared_error(y_test_con, y_pred_con)
    mse_price = mean_squared_error(y_test_price, y_pred_price)
    return mse_con, mse_price

# Funktion zum Erstellen und Trainieren des Modells
def create_and_train_model(data):
    training_data = load_historical_data(data)
    
    if training_data is not None:
        X_features, y_consumption, y_price = select_features_and_target_variables(training_data)
        
        # Variante 1: Aufteilen der Daten in Trainings- und Testmengen nach Datum
        X_train_con_v1, X_test_con_v1, y_train_con_v1, y_test_con_v1 = split_training_data_test_by_date(X_features, y_consumption, split_date='2023-09-30')
        X_train_price_v1, X_test_price_v1, y_train_price_v1, y_test_price_v1 = split_training_data_test_by_date(X_features, y_price, split_date='2023-09-30')
        
        # Variante 2: Zufällige Aufteilung der Daten in Trainings- und Testmengen
        X_train_con_v2, X_test_con_v2, y_train_con_v2, y_test_con_v2 = split_training_test_data(X_features, y_consumption, test_size=0.2, random_state=42)
        X_train_price_v2, X_test_price_v2, y_train_price_v2, y_test_price_v2 = split_training_test_data(X_features, y_price, test_size=0.2, random_state=42)
      
        # Variante 3: Kombination von zeitlicher und zufälliger Aufteilung der Daten
        X_train_con_v3, X_val_con_v3, X_test_con_v3, y_train_con_v3, y_val_con_v3, y_test_con_v3 = split_and_randomize_data(X_features, y_consumption, split_date='2023-09-30', test_size=0.2, validation_size=0.2, random_state=42)
        X_train_price_v3, X_val_price_v3, X_test_price_v3, y_train_price_v3, y_val_price_v3, y_test_price_v3 = split_and_randomize_data(X_features, y_price, split_date='2023-09-30', test_size=0.2, validation_size=0.2, random_state=42)
      
        # Trainieren der Modelle für jede Variante
        model_consumption_v1 = train_model_XGBoost(X_train_con_v1, X_test_con_v1, y_train_con_v1, y_test_con_v1)
        model_price_v1 = train_model_XGBoost(X_train_price_v1, X_test_price_v1, y_train_price_v1, y_test_price_v1)
        
        model_consumption_v2 = train_model_XGBoost(X_train_con_v2, X_test_con_v2, y_train_con_v2, y_test_con_v2)
        model_price_v2 = train_model_XGBoost(X_train_price_v2, X_test_price_v2, y_train_price_v2, y_test_price_v2)
        
        model_consumption_v3 = train_model_XGBoost(X_train_con_v3, X_test_con_v3, y_train_con_v3, y_test_con_v3)
        model_price_v3 = train_model_XGBoost(X_train_price_v3, X_test_price_v3, y_train_price_v3, y_test_price_v3)
        
        # Evaluation der Modelle für jede Variante
        mse_con_v1, mse_price_v1 = evaluate_models(y_test_con_v1, model_consumption_v1.predict(X_test_con_v1), y_test_price_v1, model_price_v1.predict(X_test_price_v1))
        mse_con_v2, mse_price_v2 = evaluate_models(y_test_con_v2, model_consumption_v2.predict(X_test_con_v2), y_test_price_v2, model_price_v2.predict(X_test_price_v2))
        mse_con_v3, mse_price_v3 = evaluate_models(y_test_con_v3, model_consumption_v3.predict(X_test_con_v3), y_test_price_v3, model_price_v3.predict(X_test_price_v3))
        
        # Auswahl des Modells mit der geringsten Abweichung
        mse_v1_avg = (mse_con_v1 + mse_price_v1) / 2
        mse_v2_avg = (mse_con_v2 + mse_price_v2) / 2
        mse_v3_avg = (mse_con_v3 + mse_price_v3) / 2
        
        if mse_v1_avg <= mse_v2_avg and mse_v1_avg <= mse_v3_avg:
            return model_consumption_v1, model_price_v1
        elif mse_v2_avg <= mse_v1_avg and mse_v2_avg <= mse_v3_avg:
            return model_consumption_v2, model_price_v2
        else:
            return model_consumption_v3, model_price_v3
    
    return None, None

# Funktion zur Erstellung von Vorhersagen für das Jahr 2024
def make_predictions_2024(model_consumption, model_price):
    # Generieren eines Datums- und Uhrzeitbereichs für das gesamte Jahr 2024 mit 15-minütigen Intervallen
    dates_2024 = pd.date_range(start='2024-01-01 00:15:00', end='2025-01-01 00:00:00', freq='15T')
    
    # Erstellen eines leeren DataFrames mit den generierten Daten und Hinzufügen von 'Zeitstempel'
    predictions_2024 = pd.DataFrame(index=dates_2024)
    predictions_2024['Zeitstempel'] = predictions_2024.index
        
    predictions_2024['Stunde'] = predictions_2024.index.hour
    predictions_2024['Tag'] = predictions_2024.index.day
    predictions_2024['Wochentag'] = predictions_2024.index.weekday
    predictions_2024['Tag des Jahres'] = predictions_2024.index.dayofyear
    predictions_2024['Kalenderwoche'] = predictions_2024.index.isocalendar().week
    predictions_2024['Monat'] = predictions_2024.index.month
    predictions_2024['Quartal'] = predictions_2024.index.quarter
    predictions_2024['Jahr'] = predictions_2024.index.year
    
    # Vorhersagen für jedes Modell unter Verwendung der Merkmale von 2024
    X_features_2024 = predictions_2024[['Stunde', 'Tag', 'Wochentag', 'Tag des Jahres', 'Kalenderwoche', 'Monat', 'Quartal', 'Jahr']]
    predictions_2024['Pred_Lasten Netz [kWh]'] = model_consumption.predict(X_features_2024)
    predictions_2024['Pred_Strommarktpreis [€/kWh]'] = model_price.predict(X_features_2024)
    
    return predictions_2024

def run_machine_learning(historical_data):
    model_consumption, model_price = create_and_train_model(historical_data)
    return make_predictions_2024(model_consumption, model_price)